## Data Preparation
1. Deleting talks with missing values
2. Cleaning up transcripts
3. Data Exploration


In [16]:
import pandas as pd
import json

import numpy as np
import matplotlib.pyplot as plt

import pickle 
import re

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import RidgeCV
sns.set_style("darkgrid")
from copy import deepcopy

%pylab inline

%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [17]:
pd.set_option('display.max_columns', 25)
pd.set_option('display.max_rows', 50)
pd.set_option('display.precision', 3)

In [18]:
df = pd.read_csv('../Data/TED_Talks_FullSet.csv', encoding='latin-1')

In [19]:
df

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,...,Dic.1,harm_virtue,harm_vice,fairness_virtue,fairness_vice,ingroup_virtue,ingroup_vice,authority_virtue,authority_vice,purity_virtue,purity_vice,morality_general
0,1.0,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2.0,2006.0,TED2006,0:16:17,6/27/06,3.177e+06,...,1.01,0.04,0.04,0.00,0.00,0.48,0.00,0.22,0.00,0.00,0.00,0.22
1,2.0,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2.0,2006.0,TED2006,0:15:06,8/15/06,1.379e+06,...,0.82,0.07,0.04,0.00,0.00,0.30,0.00,0.11,0.00,0.11,0.04,0.15
2,3.0,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7.0,2005.0,TEDGlobal 2005,0:18:45,10/18/06,7.905e+05,...,1.23,0.12,0.12,0.16,0.04,0.32,0.12,0.20,0.00,0.04,0.04,0.08
3,4.0,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2.0,2006.0,TED2006,0:19:37,10/25/06,1.985e+06,...,0.80,0.16,0.19,0.00,0.00,0.19,0.00,0.10,0.00,0.00,0.00,0.16
4,5.0,Chris Bangle,Great cars are great art,http://www.ted.com/talks/view/id/5,American designer Chris Bangle explains his ph...,http://www.ted.com/talks/view/id/5/transcript?...,2.0,2002.0,TED2002,0:20:04,4/5/07,8.595e+05,...,0.68,0.00,0.05,0.03,0.00,0.39,0.00,0.05,0.00,0.00,0.03,0.13
5,6.0,Craig Venter,Sampling the ocean's DNA,http://www.ted.com/talks/view/id/6,Genomics pioneer Craig Venter takes a break fr...,http://www.ted.com/talks/view/id/6/transcript?...,7.0,2005.0,TEDGlobal 2005,0:16:51,4/5/07,5.558e+05,...,0.90,0.00,0.08,0.08,0.00,0.27,0.04,0.31,0.00,0.04,0.00,0.08
6,7.0,David Pogue,Simplicity sells,http://www.ted.com/talks/view/id/7,New York Times columnist David Pogue takes aim...,http://www.ted.com/talks/view/id/7/transcript?...,2.0,2006.0,TED2006,0:21:26,6/27/06,1.619e+06,...,0.69,0.03,0.00,0.00,0.00,0.11,0.00,0.03,0.00,0.03,0.06,0.44
7,8.0,David Rockwell,A memorial at Ground Zero,http://www.ted.com/talks/view/id/8,In this emotionally charged conversation with ...,http://www.ted.com/talks/view/id/8/transcript?...,2.0,2002.0,TED2002,0:24:37,6/12/07,3.960e+05,...,0.97,0.05,0.07,0.02,0.00,0.45,0.00,0.17,0.00,0.17,0.00,0.02
8,9.0,Dean Kamen,To invent is to give,http://www.ted.com/talks/view/id/9,Inventor Dean Kamen lays out his argument for ...,http://www.ted.com/talks/view/id/9/transcript?...,2.0,2002.0,TED2002,0:20:07,4/5/07,6.704e+05,...,0.70,0.00,0.09,0.06,0.00,0.18,0.00,0.15,0.00,0.12,0.00,0.09
9,10.0,Dean Ornish,The killer American diet that's sweeping the p...,http://www.ted.com/talks/view/id/10,Forget the latest disease in the news: Cardiov...,http://www.ted.com/talks/view/id/10/transcript...,2.0,2006.0,TED2006,0:03:18,12/14/06,2.256e+06,...,1.91,0.16,0.16,0.16,0.00,0.16,0.00,0.16,0.00,0.00,0.96,0.16


In [20]:
listCol = list(df.columns)

In [21]:
listCol

['id',
 'speaker',
 'headline',
 'URL',
 'description',
 'transcript_URL',
 'month_filmed',
 'year_filmed',
 'event',
 'duration',
 'date_published',
 'views_as_of_06162017',
 'tags',
 'transcript',
 'notes',
 'WC',
 'Analytic',
 'Clout',
 'Authentic',
 'Tone',
 'WPS',
 'Sixltr',
 'Dic',
 'function',
 'pronoun',
 'ppron',
 'i',
 'we',
 'you',
 'shehe',
 'they',
 'ipron',
 'article',
 'prep',
 'auxverb',
 'adverb',
 'conj',
 'negate',
 'verb',
 'adj',
 'compare',
 'interrog',
 'number',
 'quant',
 'affect',
 'posemo',
 'negemo',
 'anx',
 'anger',
 'sad',
 'social',
 'family',
 'friend',
 'female',
 'male',
 'cogproc',
 'insight',
 'cause',
 'discrep',
 'tentat',
 'certain',
 'differ',
 'percept',
 'see',
 'hear',
 'feel',
 'bio',
 'body',
 'health',
 'sexual',
 'ingest',
 'drives',
 'affiliation',
 'achieve',
 'power',
 'reward',
 'risk',
 'focuspast',
 'focuspresent',
 'focusfuture',
 'relativ',
 'motion',
 'space',
 'time',
 'work',
 'leisure',
 'home',
 'money',
 'relig',
 'death',
 

#### Storing only columns that are information fields from TED

In [22]:
Data = df.filter(['id',
 'speaker',
 'headline',
 'URL',
 'description',
 'transcript_URL',
 'month_filmed',
 'year_filmed',
 'event',
 'duration',
 'date_published',
 'views_as_of_06162017',
 'tags',
 'transcript'])

In [23]:
Data

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript
0,1.0,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2.0,2006.0,TED2006,0:16:17,6/27/06,3.177e+06,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ..."
1,2.0,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2.0,2006.0,TED2006,0:15:06,8/15/06,1.379e+06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ..."
2,3.0,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7.0,2005.0,TEDGlobal 2005,0:18:45,10/18/06,7.905e+05,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r..."
3,4.0,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2.0,2006.0,TED2006,0:19:37,10/25/06,1.985e+06,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst..."
4,5.0,Chris Bangle,Great cars are great art,http://www.ted.com/talks/view/id/5,American designer Chris Bangle explains his ph...,http://www.ted.com/talks/view/id/5/transcript?...,2.0,2002.0,TED2002,0:20:04,4/5/07,8.595e+05,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac..."
5,6.0,Craig Venter,Sampling the ocean's DNA,http://www.ted.com/talks/view/id/6,Genomics pioneer Craig Venter takes a break fr...,http://www.ted.com/talks/view/id/6/transcript?...,7.0,2005.0,TEDGlobal 2005,0:16:51,4/5/07,5.558e+05,"biotech,invention,oceans,genetics,DNA,biology,...","0:11\r\r\rAt the break, I was asked by several..."
6,7.0,David Pogue,Simplicity sells,http://www.ted.com/talks/view/id/7,New York Times columnist David Pogue takes aim...,http://www.ted.com/talks/view/id/7/transcript?...,2.0,2006.0,TED2006,0:21:26,6/27/06,1.619e+06,"simplicity,computers,software,interface design...","0:12\r\r\r(Music: ""The Sound of Silence,""\rSim..."
7,8.0,David Rockwell,A memorial at Ground Zero,http://www.ted.com/talks/view/id/8,In this emotionally charged conversation with ...,http://www.ted.com/talks/view/id/8/transcript?...,2.0,2002.0,TED2002,0:24:37,6/12/07,3.960e+05,"New York,memory,interview,death,culture,archit...","0:13\r\r\rKurt Andersen: Like many architects,..."
8,9.0,Dean Kamen,To invent is to give,http://www.ted.com/talks/view/id/9,Inventor Dean Kamen lays out his argument for ...,http://www.ted.com/talks/view/id/9/transcript?...,2.0,2002.0,TED2002,0:20:07,4/5/07,6.704e+05,"robots,cars,industrial design,transportation,i...","0:11\r\r\rAs you pointed out, every time you c..."
9,10.0,Dean Ornish,The killer American diet that's sweeping the p...,http://www.ted.com/talks/view/id/10,Forget the latest disease in the news: Cardiov...,http://www.ted.com/talks/view/id/10/transcript...,2.0,2006.0,TED2006,0:03:18,12/14/06,2.256e+06,"obesity,disease,health,health care,culture,foo...",0:11\r\r\rWith all the legitimate concerns\rab...


In [24]:
Data.dtypes

id                      float64
speaker                  object
headline                 object
URL                      object
description              object
transcript_URL           object
month_filmed            float64
year_filmed             float64
event                    object
duration                 object
date_published           object
views_as_of_06162017    float64
tags                     object
transcript               object
dtype: object

### Clean up Missing Transcipts

In [25]:
#Drop the rows where transcript is null
len(Data[(Data['transcript'].isnull())] ) 

89

In [26]:
mask = Data['transcript'].isnull()

In [27]:
Data = Data[~mask]

In [28]:
Data.columns

Index(['id', 'speaker', 'headline', 'URL', 'description', 'transcript_URL',
       'month_filmed', 'year_filmed', 'event', 'duration', 'date_published',
       'views_as_of_06162017', 'tags', 'transcript'],
      dtype='object')

In [29]:
len(Data[(Data['transcript'].isnull())] ) 

0

In [30]:
Data['tags'].head(5)

0    cars,alternative energy,culture,politics,scien...
1    MacArthur grant,simplicity,industrial design,a...
2    corruption,poverty,economics,investment,milita...
3    aircraft,flight,industrial design,NASA,rocket ...
4    cars,industrial design,transportation,inventio...
Name: tags, dtype: object

In [31]:
Data['headline'].head(5)

0             Averting the climate crisis
1           Simple designs to save a life
2           How to rebuild a broken state
3    The real future of space exploration
4                Great cars are great art
Name: headline, dtype: object

#### Cleaning Up more rows

In [32]:
Data.year_filmed.unique()

array([ 2006.  ,  2005.  ,  2002.  ,  2003.  ,  2004.  ,  2007.  ,
        2008.  ,  1984.  ,  1990.  ,  1998.  ,  2001.  ,    17.75,
        2009.  ,  2010.  ,  2011.  ,  2012.  ,  1994.  ,  2013.  ,
        2014.  ,  2015.  ,  2016.  ,  2017.  ,    18.41,    17.72,    15.1 ])

In [33]:
Data[(Data['year_filmed']==18.41)]

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript
2398,NaN,Transcript extended into text file,5800,55.87,68.65,48.28,23.87,18.41,17.91,85.55,53.6,12.86,5.97,1.5


In [34]:
Data[(Data['year_filmed']== 17.72)]

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript
2418,NaN,Transcript extended into text file,5918,44.26,70.75,33.15,49.68,17.72,16.98,85.97,56.37,15.92,8.33,2.91


In [35]:
Data[(Data['year_filmed']==  15.1)]

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript
2452,NaN,Transcript extended into text file,6008,51.03,66.21,34.91,68.2,15.1,13.95,79.91,51.71,13.67,5.96,1.45


In [36]:
Data[(Data['year_filmed']==  17.75)]

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript
350,NaN,Transcript extended into text file,6088,47.47,82.8,30.58,37.09,17.75,14.41,88.26,56.01,17.43,8.62,1.53


Deleting the above rows of data

In [37]:
Data = Data[Data.year_filmed != 18.41]
Data = Data[Data.year_filmed != 17.72]
Data = Data[Data.year_filmed != 15.1]
Data = Data[Data.year_filmed != 17.75]


#### Checking if rows were deleted

In [38]:
Data.year_filmed.unique()

array([ 2006.,  2005.,  2002.,  2003.,  2004.,  2007.,  2008.,  1984.,
        1990.,  1998.,  2001.,  2009.,  2010.,  2011.,  2012.,  1994.,
        2013.,  2014.,  2015.,  2016.,  2017.])

In [39]:
Data[Data['year_filmed']==  1984.]

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript
206,230.0,Nicholas Negroponte,"5 predictions, from 1984",http://www.ted.com/talks/view/id/230,"With surprising accuracy, Nicholas Negroponte ...",http://www.ted.com/talks/view/id/230/transcrip...,2.0,1984.0,TED1984,0:25:23,3/11/08,963720.0,"interface design,education,demo,media,future,d...",0:11\r\r\rIn this rather long sort of marathon...


### Reset Index

In [40]:
Data = Data.reset_index(drop=True)

## Data Exploration

### No. of Talks

In [42]:
Data.shape

(2386, 14)

### No. of unique speakers

In [43]:
Data.speaker.nunique()

2017

#### No. of events

In [44]:
Data.event.nunique()

314

#### No.of Years

In [45]:
Data.year_filmed.nunique()

21

In [46]:
Data.year_filmed.unique()

array([ 2006.,  2005.,  2002.,  2003.,  2004.,  2007.,  2008.,  1984.,
        1990.,  1998.,  2001.,  2009.,  2010.,  2011.,  2012.,  1994.,
        2013.,  2014.,  2015.,  2016.,  2017.])

#### No. of Talks by Actual Year of Talk 

In [47]:
Data.groupby('year_filmed').id.count()

year_filmed
1984.0      1
1990.0      1
1994.0      1
1998.0      6
2001.0      3
2002.0     26
2003.0     35
2004.0     32
2005.0     62
2006.0     47
2007.0    110
2008.0     86
2009.0    212
2010.0    259
2011.0    237
2012.0    258
2013.0    268
2014.0    236
2015.0    237
2016.0    230
2017.0     39
Name: id, dtype: int64

### Date/Year Published Analysis

In [48]:
import datetime as dt

In [49]:
Data['date_published'] = pd.to_datetime(Data['date_published'])

In [50]:
#check change
Data.date_published.dtypes

dtype('<M8[ns]')

In [51]:
# Creating columns for date and month of publications
Data['Pub_Year'] = Data['date_published'].dt.year
Data['Pub_Month'] = Data['date_published'].dt.month

In [52]:
Data.head()

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
0,1.0,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2.0,2006.0,TED2006,0:16:17,2006-06-27,3.177e+06,"cars,alternative energy,culture,politics,scien...","0:14\r\r\rThank you so much, Chris.\rAnd it's ...",2006,6
1,2.0,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2.0,2006.0,TED2006,0:15:06,2006-08-15,1.379e+06,"MacArthur grant,simplicity,industrial design,a...","0:11\r\r\rIn terms of invention,\rI'd like to ...",2006,8
2,3.0,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7.0,2005.0,TEDGlobal 2005,0:18:45,2006-10-18,7.905e+05,"corruption,poverty,economics,investment,milita...","0:12\r\r\rA public, Dewey long ago observed,\r...",2006,10
3,4.0,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2.0,2006.0,TED2006,0:19:37,2006-10-25,1.985e+06,"aircraft,flight,industrial design,NASA,rocket ...","0:11\r\r\rI want to start off by saying, Houst...",2006,10
4,5.0,Chris Bangle,Great cars are great art,http://www.ted.com/talks/view/id/5,American designer Chris Bangle explains his ph...,http://www.ted.com/talks/view/id/5/transcript?...,2.0,2002.0,TED2002,0:20:04,2007-04-05,8.595e+05,"cars,industrial design,transportation,inventio...","0:12\r\r\rWhat I want to talk about is, as bac...",2007,4


### No. of talks published by Year

In [53]:
Data.groupby('Pub_Year').id.count()

Pub_Year
2006     50
2007    119
2008    185
2009    215
2010    233
2011    246
2012    275
2013    241
2014    240
2015    220
2016    243
2017    119
Name: id, dtype: int64

## Removing Time (5:30), \r,\n,(Laughter),(Applause)

In [54]:
Data['transcript'].head(2)

0    0:14\r\r\rThank you so much, Chris.\rAnd it's ...
1    0:11\r\r\rIn terms of invention,\rI'd like to ...
Name: transcript, dtype: object

Use http://www.rubular.com/r/JsdNM3nFJ3


In [55]:
text =Data['transcript'].head(1)

In [56]:
text[0] 

'0:14\r\r\rThank you so much, Chris.\rAnd it\'s truly a great honor\rto have the opportunity\rto come to this stage twice;\rI\'m extremely grateful.\rI have been blown away by this conference,\rand I want to thank all of you\rfor the many nice comments\rabout what I had to say the other night.\rAnd I say that sincerely,\rpartly because (Mock sob)\rI need that.\r\r\r\r\r 0:40\r\r\r(Laughter)\r\r\r\r\r 0:45\r\r\rPut yourselves in my position.\r\r\r\r\r 0:47\r\r\r(Laughter)\r\r\r\r\r 0:54\r\r\rI flew on Air Force Two for eight years.\r\r\r\r\r 0:57\r\r\r(Laughter)\r\r\r\r\r 0:59\r\r\rNow I have to take off my shoes\ror boots to get on an airplane!\r\r\r\r\r 1:02\r\r\r(Laughter)\r\r\r\r\r 1:05\r\r\r(Applause)\r\r\r\r\r 1:11\r\r\rI\'ll tell you one quick story\rto illustrate what\rthat\'s been like for me.\r\r\r\r\r 1:16\r\r\r(Laughter)\r\r\r\r\r 1:18\r\r\rIt\'s a true story \revery bit of this is true.\r\r\r\r\r 1:21\r\r\rSoon after Tipper and I left the \r(Mock sob) White House \r\r\r\r\r

r'(\d{1,2}:\d{2})' Explanation for removing time (eg. 2:41 or 15:20)  
( ) - for exact pattern match --> matches all elements in the exact order within ( )  
\d{1,2} - digits between 1 to 2 digits  
: - digits followed by :  
\d{2} -  Exactly 2 digits after :  

In [57]:
def StripTranscript(x):
        p = re.compile(r'(\d{1,2}:\d{2})')
        a = p.sub('', x)
        a=a.replace('\n',' ')
       
        #a=a.replace("'","") # don't include as it is included in stop words
        a=a.replace('(Laughter)', ' ')
        a=a.replace('(Applause)', ' ')
        lista = a.split()
        for i in range(len(lista)):
            if lista[i] =='\r':
                if lista[i+1]=='\r':
                    lista[i]=lista[i].replace('\r','')
                else:
                    lista[i]=lista[i].replace('\r',' ')      
        a=" ".join(lista)
        return  a


In [58]:
StripTranscript(text[0])

'Thank you so much, Chris. And it\'s truly a great honor to have the opportunity to come to this stage twice; I\'m extremely grateful. I have been blown away by this conference, and I want to thank all of you for the many nice comments about what I had to say the other night. And I say that sincerely, partly because (Mock sob) I need that. Put yourselves in my position. I flew on Air Force Two for eight years. Now I have to take off my shoes or boots to get on an airplane! I\'ll tell you one quick story to illustrate what that\'s been like for me. It\'s a true story every bit of this is true. Soon after Tipper and I left the (Mock sob) White House we were driving from our home in Nashville to a little farm we have 50 miles east of Nashville. Driving ourselves. I know it sounds like a little thing to you, but I looked in the rear-view mirror and all of a sudden it just hit me. There was no motorcade back there. You\'ve heard of phantom limb pain? This was a rented Ford Taurus. It was di

In [59]:
StripTranscript(text[0][200:300])

'k all of you for the many nice comments about what I had to say the other night. And I say that sinc'

In [60]:
Data['transcript'] = Data['transcript'].map(StripTranscript)

In [61]:
Data['transcript'].head(5)

0    Thank you so much, Chris. And it's truly a gre...
1    In terms of invention, I'd like to tell you th...
2    A public, Dewey long ago observed, is constitu...
3    I want to start off by saying, Houston, we hav...
4    What I want to talk about is, as background, i...
Name: transcript, dtype: object

#### Correcting A Headline

In [85]:
Data[Data['headline']=='WhatÛªs wrong with your pa$$w0rd?']

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
1692,2030.0,Lorrie Faith Cranor,WhatÛªs wrong with your pa$$w0rd?,http://www.ted.com/talks/view/id/2030,Lorrie Faith Cranor studied thousands of real ...,http://www.ted.com/talks/view/id/2030/transcri...,3.0,2014.0,TEDxCMU,0:17:41,2014-06-24,1.453e+06,"privacy,hack,TEDx,security,technology,business...",I am a computer science and engineering profes...,2014,6


In [86]:
Data = Data.replace('WhatÛªs wrong with your pa$$w0rd?', 'What\'s wrong with your password?')

In [88]:
Data[Data['headline']=='What\'s wrong with your password?']

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
1692,2030.0,Lorrie Faith Cranor,What's wrong with your password?,http://www.ted.com/talks/view/id/2030,Lorrie Faith Cranor studied thousands of real ...,http://www.ted.com/talks/view/id/2030/transcri...,3.0,2014.0,TEDxCMU,0:17:41,2014-06-24,1.453e+06,"privacy,hack,TEDx,security,technology,business...",I am a computer science and engineering profes...,2014,6


In [89]:
import pickle 

In [90]:
with open('../Data/DataDF.pkl', 'wb') as picklefile:
    pickle.dump(Data, picklefile)

In [91]:
with open('../Data/DataDF.pkl', 'rb') as picklefile5:
    Data2 = pickle.load(picklefile5)


In [92]:
Data2

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
0,1.0,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2.0,2006.0,TED2006,0:16:17,2006-06-27,3.177e+06,"cars,alternative energy,culture,politics,scien...","Thank you so much, Chris. And it's truly a gre...",2006,6
1,2.0,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2.0,2006.0,TED2006,0:15:06,2006-08-15,1.379e+06,"MacArthur grant,simplicity,industrial design,a...","In terms of invention, I'd like to tell you th...",2006,8
2,3.0,Ashraf Ghani,How to rebuild a broken state,http://www.ted.com/talks/view/id/3,Ashraf Ghani's passionate and powerful 10-minu...,http://www.ted.com/talks/view/id/3/transcript?...,7.0,2005.0,TEDGlobal 2005,0:18:45,2006-10-18,7.905e+05,"corruption,poverty,economics,investment,milita...","A public, Dewey long ago observed, is constitu...",2006,10
3,4.0,Burt Rutan,The real future of space exploration,http://www.ted.com/talks/view/id/4,"In this passionate talk, legendary spacecraft ...",http://www.ted.com/talks/view/id/4/transcript?...,2.0,2006.0,TED2006,0:19:37,2006-10-25,1.985e+06,"aircraft,flight,industrial design,NASA,rocket ...","I want to start off by saying, Houston, we hav...",2006,10
4,5.0,Chris Bangle,Great cars are great art,http://www.ted.com/talks/view/id/5,American designer Chris Bangle explains his ph...,http://www.ted.com/talks/view/id/5/transcript?...,2.0,2002.0,TED2002,0:20:04,2007-04-05,8.595e+05,"cars,industrial design,transportation,inventio...","What I want to talk about is, as background, i...",2007,4
5,6.0,Craig Venter,Sampling the ocean's DNA,http://www.ted.com/talks/view/id/6,Genomics pioneer Craig Venter takes a break fr...,http://www.ted.com/talks/view/id/6/transcript?...,7.0,2005.0,TEDGlobal 2005,0:16:51,2007-04-05,5.558e+05,"biotech,invention,oceans,genetics,DNA,biology,...","At the break, I was asked by several people ab...",2007,4
6,7.0,David Pogue,Simplicity sells,http://www.ted.com/talks/view/id/7,New York Times columnist David Pogue takes aim...,http://www.ted.com/talks/view/id/7/transcript?...,2.0,2006.0,TED2006,0:21:26,2006-06-27,1.619e+06,"simplicity,computers,software,interface design...","(Music: ""The Sound of Silence,"" Simon & Garfun...",2006,6
7,8.0,David Rockwell,A memorial at Ground Zero,http://www.ted.com/talks/view/id/8,In this emotionally charged conversation with ...,http://www.ted.com/talks/view/id/8/transcript?...,2.0,2002.0,TED2002,0:24:37,2007-06-12,3.960e+05,"New York,memory,interview,death,culture,archit...","Kurt Andersen: Like many architects, David is ...",2007,6
8,9.0,Dean Kamen,To invent is to give,http://www.ted.com/talks/view/id/9,Inventor Dean Kamen lays out his argument for ...,http://www.ted.com/talks/view/id/9/transcript?...,2.0,2002.0,TED2002,0:20:07,2007-04-05,6.704e+05,"robots,cars,industrial design,transportation,i...","As you pointed out, every time you come here, ...",2007,4
9,10.0,Dean Ornish,The killer American diet that's sweeping the p...,http://www.ted.com/talks/view/id/10,Forget the latest disease in the news: Cardiov...,http://www.ted.com/talks/view/id/10/transcript...,2.0,2006.0,TED2006,0:03:18,2006-12-14,2.256e+06,"obesity,disease,health,health care,culture,foo...",With all the legitimate concerns about AIDS an...,2006,12


# Popularity Exploration

## Sorting the talks by the no. of views

#### Sorting by No. of Views (High to low)

In [93]:
Data.sort_values(by=['views_as_of_06162017'],ascending=False)

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
61,66.0,Ken Robinson,Do schools kill creativity?,http://www.ted.com/talks/view/id/66,Sir Ken Robinson makes an entertaining and pro...,http://www.ted.com/talks/view/id/66/transcript...,2.0,2006.0,TED2006,0:19:24,2006-06-27,4.562e+07,"dance,education,culture,parenting,children,tea...","Good morning. How are you? It's been great, ha...",2006,6
1260,1569.0,Amy Cuddy,Your body language shapes who you are,http://www.ted.com/talks/view/id/1569,"Body language affects how others see us, but i...",http://www.ted.com/talks/view/id/1569/transcri...,6.0,2012.0,TEDGlobal 2012,0:21:02,2012-10-01,4.151e+07,"TED Brain Trust,success,body language,psycholo...",So I want to start by offering you a free no-t...,2012,10
650,848.0,Simon Sinek,How great leaders inspire action,http://www.ted.com/talks/view/id/848,Simon Sinek has a simple but powerful model fo...,http://www.ted.com/talks/view/id/848/transcrip...,9.0,2009.0,TEDxPuget Sound,0:18:04,2010-05-04,3.266e+07,"success,TEDx,leadership,entrepreneur,business",How do you explain when things don't go as we ...,2010,5
801,1042.0,BrenÌ© Brown,The power of vulnerability,http://www.ted.com/talks/view/id/1042,BrenÌ© Brown studies human connection -- our a...,http://www.ted.com/talks/view/id/1042/transcri...,6.0,2010.0,TEDxHouston,0:20:19,2010-12-23,2.993e+07,"depression,mental health,psychology,self,TEDx,...","So, I'll start with this: a couple years ago, ...",2010,12
438,549.0,Mary Roach,10 things you didn't know about orgasm,http://www.ted.com/talks/view/id/549,"""Bonk"" author Mary Roach delves into obscure s...",http://www.ted.com/talks/view/id/549/transcrip...,2.0,2009.0,TED2009,0:16:43,2009-05-20,2.190e+07,"sex,books,culture,history,science,humor,writing",Alright. I'm going to show you a couple of ima...,2009,5
201,229.0,Jill Bolte Taylor,My stroke of insight,http://www.ted.com/talks/view/id/229,Jill Bolte Taylor got a research opportunity f...,http://www.ted.com/talks/view/id/229/transcrip...,2.0,2008.0,TED2008,0:18:19,2008-03-12,2.064e+07,"consciousness,illness,biology,brain,science,gl...",I grew up to study the brain because I have a ...,2008,3
89,96.0,Tony Robbins,Why we do what we do,http://www.ted.com/talks/view/id/96,"Tony Robbins discusses the ""invisible forces"" ...",http://www.ted.com/talks/view/id/96/transcript...,2.0,2006.0,TED2006,0:21:45,2006-06-27,1.998e+07,"goal-setting,psychology,motivation,culture,pot...",Thank you. I have to tell you I'm both challen...,2006,6
1696,2034.0,Julian Treasure,How to speak so that people want to listen,http://www.ted.com/talks/view/id/2034,"Have you ever felt like you're talking, but no...",http://www.ted.com/talks/view/id/2034/transcri...,6.0,2013.0,TEDGlobal 2013,0:09:58,2014-06-27,1.951e+07,"sound,culture,speech",The human voice: It's the instrument we all pl...,2014,6
1333,1647.0,Cameron Russell,"Looks aren't everything. Believe me, I'm a model.",http://www.ted.com/talks/view/id/1647,"Cameron Russell admits she won ""a genetic lott...",http://www.ted.com/talks/view/id/1647/transcri...,10.0,2012.0,TEDxMidAtlantic,0:09:37,2013-01-16,1.881e+07,"fashion,photography,TEDx,culture,beauty","Hi. My name is Cameron Russell, and for the la...",2013,1
485,618.0,Dan Pink,The puzzle of motivation,http://www.ted.com/talks/view/id/618,Career analyst Dan Pink examines the puzzle of...,http://www.ted.com/talks/view/id/618/transcrip...,7.0,2009.0,TEDGlobal 2009,0:18:36,2009-08-24,1.825e+07,"brain,motivation,social change,science,work,cr...",I need to make a confession at the outset here...,2009,8


### Top 25

In [94]:
top25 = Data.sort_values(by=['views_as_of_06162017'],ascending=False).head(25)

In [95]:
top25

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
61,66.0,Ken Robinson,Do schools kill creativity?,http://www.ted.com/talks/view/id/66,Sir Ken Robinson makes an entertaining and pro...,http://www.ted.com/talks/view/id/66/transcript...,2.0,2006.0,TED2006,0:19:24,2006-06-27,4.562e+07,"dance,education,culture,parenting,children,tea...","Good morning. How are you? It's been great, ha...",2006,6
1260,1569.0,Amy Cuddy,Your body language shapes who you are,http://www.ted.com/talks/view/id/1569,"Body language affects how others see us, but i...",http://www.ted.com/talks/view/id/1569/transcri...,6.0,2012.0,TEDGlobal 2012,0:21:02,2012-10-01,4.151e+07,"TED Brain Trust,success,body language,psycholo...",So I want to start by offering you a free no-t...,2012,10
650,848.0,Simon Sinek,How great leaders inspire action,http://www.ted.com/talks/view/id/848,Simon Sinek has a simple but powerful model fo...,http://www.ted.com/talks/view/id/848/transcrip...,9.0,2009.0,TEDxPuget Sound,0:18:04,2010-05-04,3.266e+07,"success,TEDx,leadership,entrepreneur,business",How do you explain when things don't go as we ...,2010,5
801,1042.0,BrenÌ© Brown,The power of vulnerability,http://www.ted.com/talks/view/id/1042,BrenÌ© Brown studies human connection -- our a...,http://www.ted.com/talks/view/id/1042/transcri...,6.0,2010.0,TEDxHouston,0:20:19,2010-12-23,2.993e+07,"depression,mental health,psychology,self,TEDx,...","So, I'll start with this: a couple years ago, ...",2010,12
438,549.0,Mary Roach,10 things you didn't know about orgasm,http://www.ted.com/talks/view/id/549,"""Bonk"" author Mary Roach delves into obscure s...",http://www.ted.com/talks/view/id/549/transcrip...,2.0,2009.0,TED2009,0:16:43,2009-05-20,2.190e+07,"sex,books,culture,history,science,humor,writing",Alright. I'm going to show you a couple of ima...,2009,5
201,229.0,Jill Bolte Taylor,My stroke of insight,http://www.ted.com/talks/view/id/229,Jill Bolte Taylor got a research opportunity f...,http://www.ted.com/talks/view/id/229/transcrip...,2.0,2008.0,TED2008,0:18:19,2008-03-12,2.064e+07,"consciousness,illness,biology,brain,science,gl...",I grew up to study the brain because I have a ...,2008,3
89,96.0,Tony Robbins,Why we do what we do,http://www.ted.com/talks/view/id/96,"Tony Robbins discusses the ""invisible forces"" ...",http://www.ted.com/talks/view/id/96/transcript...,2.0,2006.0,TED2006,0:21:45,2006-06-27,1.998e+07,"goal-setting,psychology,motivation,culture,pot...",Thank you. I have to tell you I'm both challen...,2006,6
1696,2034.0,Julian Treasure,How to speak so that people want to listen,http://www.ted.com/talks/view/id/2034,"Have you ever felt like you're talking, but no...",http://www.ted.com/talks/view/id/2034/transcri...,6.0,2013.0,TEDGlobal 2013,0:09:58,2014-06-27,1.951e+07,"sound,culture,speech",The human voice: It's the instrument we all pl...,2014,6
1333,1647.0,Cameron Russell,"Looks aren't everything. Believe me, I'm a model.",http://www.ted.com/talks/view/id/1647,"Cameron Russell admits she won ""a genetic lott...",http://www.ted.com/talks/view/id/1647/transcri...,10.0,2012.0,TEDxMidAtlantic,0:09:37,2013-01-16,1.881e+07,"fashion,photography,TEDx,culture,beauty","Hi. My name is Cameron Russell, and for the la...",2013,1
485,618.0,Dan Pink,The puzzle of motivation,http://www.ted.com/talks/view/id/618,Career analyst Dan Pink examines the puzzle of...,http://www.ted.com/talks/view/id/618/transcrip...,7.0,2009.0,TEDGlobal 2009,0:18:36,2009-08-24,1.825e+07,"brain,motivation,social change,science,work,cr...",I need to make a confession at the outset here...,2009,8


In [96]:
with open('../Data/Top25.pkl', 'wb') as picklefile2:
    pickle.dump(top25, picklefile2)

#### Bottom25

In [97]:
bottom25 = Data.sort_values(by=['views_as_of_06162017'],ascending=False).tail(25)
bottom25

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
1158,1452.0,Michael McDaniel,"Cheap, effective shelter for disaster relief",http://www.ted.com/talks/view/id/1452,Michael McDaniel designed housing for disaster...,http://www.ted.com/talks/view/id/1452/transcri...,2.0,2012.0,TEDxAustin,0:07:50,2012-05-19,267626.0,"natural disaster,TEDx,disaster relief,design,g...","So, I'm going to start off with kind of the bu...",2012,5
1035,1294.0,Srdja Popovic,How to topple a dictator,http://www.ted.com/talks/view/id/1294,People-powered resistance: can it work? Srdja ...,http://www.ted.com/talks/view/id/1294/transcri...,10.0,2011.0,TEDxKrakow,0:12:02,2011-12-05,267283.0,"protests,TEDx,culture,activism,politics,global...","Good afternoon, I'm proud to be here at TEDxKr...",2011,12
532,674.0,Dayananda Saraswati,The profound journey of compassion,http://www.ted.com/talks/view/id/674,Swami Dayananda Saraswati unravels the paralle...,http://www.ted.com/talks/view/id/674/transcrip...,8.0,2008.0,Chautauqua Institution,0:16:54,2008-10-31,264392.0,"charter for compassion,self,compassion,religio...","A human child is born, and for quite a long ti...",2008,10
1152,1445.0,Bart Knols,3 new ways to kill mosquitoes,http://www.ted.com/talks/view/id/1445,We can use a mosquito's own instincts against ...,http://www.ted.com/talks/view/id/1445/transcri...,4.0,2012.0,TEDxMaastricht,0:10:20,2012-05-13,255544.0,"insects,health,medicine,TEDx,global issues",(Mosquito buzzing) (Swat) Gotcha. Mosquitos. I...,2012,5
322,394.0,Lee Smolin,Science and democracy,http://www.ted.com/talks/view/id/394,Physicist Lee Smolin talks about how the scien...,http://www.ted.com/talks/view/id/394/transcrip...,2.0,2003.0,TED2003,0:12:25,2008-11-10,253557.0,"String theory,physics,politics,social change,s...","So, about three years ago I was in London, and...",2008,11
282,339.0,Peter Hirshberg,"The web is more than ""better TV""",http://www.ted.com/talks/view/id/339,In this absorbing look at emerging media and t...,http://www.ted.com/talks/view/id/339/transcrip...,12.0,2007.0,EG 2007,0:31:39,2008-09-04,248353.0,"telecom,media,history,technology,business","Well, good morning. You know, the computer and...",2008,9
402,500.0,C.K. Williams,Poetry of youth and age,http://www.ted.com/talks/view/id/500,Poet C.K. Williams reads his work at TED2001. ...,http://www.ted.com/talks/view/id/500/transcrip...,2.0,2001.0,TED2001,0:23:17,2009-03-30,242208.0,"poetry,literature,culture,entertainment,storyt...",I thought I would read poems I have that relat...,2009,3
327,402.0,Stewart Brand,The Long Now,http://www.ted.com/talks/view/id/402,Stewart Brand works on the Clock of the Long N...,http://www.ted.com/talks/view/id/402/transcrip...,2.0,2004.0,TED2004,0:23:23,2008-11-17,240414.0,"TED Brain Trust,architecture,future,design,tec...","Welcome to 10,000 feet. Let me explain why we ...",2008,11
349,427.0,John Maeda,My journey in design,http://www.ted.com/talks/view/id/427,Designer John Maeda talks about his path from ...,http://www.ted.com/talks/view/id/427/transcrip...,3.0,2008.0,Serious Play 2008,0:17:06,2009-01-06,237990.0,"programming,code,computers,education,personal ...","I'm kind of tired of talking about simplicity,...",2009,1
739,962.0,Mitchell Besser,Mothers helping mothers fight HIV,http://www.ted.com/talks/view/id/962,"In sub-Saharan Africa, HIV infections are more...",http://www.ted.com/talks/view/id/962/transcrip...,7.0,2010.0,TEDGlobal 2010,0:18:30,2010-09-22,231703.0,"AIDS,Africa,health,health care,women,community",I want you to take a trip with me. Picture you...,2010,9


In [98]:
with open('../Data/Bottom25.pkl', 'wb') as picklefile3:
    pickle.dump(bottom25, picklefile3)

## Top 5 Most Viewed for each Year

In [99]:
Data.sort_values(by=['Pub_Year','views_as_of_06162017'],ascending=False).groupby('Pub_Year').head(5)

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
2359,2774.0,Elon Musk,The future we're building -- and boring,http://www.ted.com/talks/view/id/2774,Elon Musk discusses his new project digging tu...,http://www.ted.com/talks/view/id/2738/transcri...,4.0,2017.0,TED2017,0:40:50,2017-04-30,3.913e+06,"environment,science,technology,design,engineer...","Chris Anderson: Elon, hey, welcome back to TED...",2017,4
2292,2678.0,Thordis Elva and Tom Stranger,Our story of rape and reconciliation,http://www.ted.com/talks/view/id/2678,"In 1996, Thordis Elva shared a teenage romance...",http://www.ted.com/talks/view/id/2679/transcri...,10.0,2016.0,TEDWomen 2016,0:19:06,2017-02-07,3.575e+06,"collaboration,communication,women,identity,sex...",[This talk contains graphic language and descr...,2017,2
2365,2780.0,Shah Rukh Khan,"Thoughts on humanity, fame and love",http://www.ted.com/talks/view/id/2780,"""I sell dreams, and I peddle love to millions ...",http://www.ted.com/talks/view/id/2742/transcri...,4.0,2017.0,TED2017,0:17:51,2017-05-12,3.539e+06,"entertainment,music,art,love,dance,aging,happi...","Namaskar. I'm a movie star, I'm 51 years of ag...",2017,5
2316,2703.0,Megan Phelps-Roper,I grew up in the Westboro Baptist Church. Here...,http://www.ted.com/talks/view/id/2703,What's it like to grow up within a group of pe...,http://www.ted.com/talks/view/id/2697/transcri...,2.0,2017.0,TEDNYC,0:15:17,2017-03-06,3.263e+06,"faith,religion,peace,motivation,compassion,emp...","I was a blue-eyed, chubby-cheeked five-year-ol...",2017,3
2268,2652.0,Adam Grant,Are you a giver or a taker?,http://www.ted.com/talks/view/id/2652,"In every workplace, there are three basic kind...",http://www.ted.com/talks/view/id/2655/transcri...,11.0,2016.0,TED@IBM,0:13:28,2017-01-03,3.077e+06,"psychology,anthropology,behavioral economics,b...",I want you to look around the room for a minut...,2017,1
2032,2405.0,James Veitch,This is what happens when you reply to spam email,http://www.ted.com/talks/view/id/2405,"Suspicious emails: unclaimed insurance bonds, ...",http://www.ted.com/talks/view/id/2405/transcri...,12.0,2015.0,TEDGlobal>Geneva,0:09:48,2016-01-08,1.775e+07,"comedy,curiosity,humor,technology,communication","A few years ago, I got one of those spam email...",2016,1
2079,2458.0,Tim Urban,Inside the mind of a master procrastinator,http://www.ted.com/talks/view/id/2458,Tim Urban knows that procrastination doesn't m...,http://www.ted.com/talks/view/id/2458/transcri...,2.0,2016.0,TED2016,0:14:03,2016-03-15,1.306e+07,"online video,productivity,goal-setting,mind,vi...","So in college, I was a government major, which...",2016,3
2059,2435.0,Celeste Headlee,10 ways to have a better conversation,http://www.ted.com/talks/view/id/2435,When your job hinges on how well you talk to p...,http://www.ted.com/talks/view/id/2435/transcri...,5.0,2015.0,TEDxCreativeCoast,0:11:44,2016-02-16,7.657e+06,"interview,TEDx,personal growth,language,commun...","All right, I want to see a show of hands: how ...",2016,2
2046,2420.0,Judson Brewer,A simple way to break a bad habit,http://www.ted.com/talks/view/id/2420,Can we break bad habits by being more curious ...,http://www.ted.com/talks/view/id/2420/transcri...,11.0,2015.0,TEDMED 2015,0:09:24,2016-02-03,7.328e+06,"obesity,mindfulness,addiction,TEDMED,cognitive...","When I was first learning to meditate, the ins...",2016,2
2095,2474.0,Adam Grant,The surprising habits of original thinkers,http://www.ted.com/talks/view/id/2474,How do creative people come up with great idea...,http://www.ted.com/talks/view/id/2474/transcri...,2.0,2016.0,TED2016,0:15:25,2016-04-01,5.788e+06,"success,invention,failure,motivation,decision-...","Seven years ago, a student came to me and aske...",2016,4


In [100]:
YearlyTop5 = Data.sort_values(by=['Pub_Year','views_as_of_06162017'],ascending=False).groupby('Pub_Year').head(5)

In [101]:
with open('../Data/YearlyTop5.pkl', 'wb') as picklefile4:
    pickle.dump(YearlyTop5, picklefile4)

# Checks

In [102]:
with open('../Data/DataDF.pkl', 'rb') as picklefile5:
    Data2 = pickle.load(picklefile5)

### Check DF

In [103]:
Data2.head(2)

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
0,1.0,Al Gore,Averting the climate crisis,http://www.ted.com/talks/view/id/1,With the same humor and humanity he exuded in ...,http://www.ted.com/talks/view/id/1/transcript?...,2.0,2006.0,TED2006,0:16:17,2006-06-27,3.177e+06,"cars,alternative energy,culture,politics,scien...","Thank you so much, Chris. And it's truly a gre...",2006,6
1,2.0,Amy Smith,Simple designs to save a life,http://www.ted.com/talks/view/id/2,Fumes from indoor cooking fires kill more than...,http://www.ted.com/talks/view/id/2/transcript?...,2.0,2006.0,TED2006,0:15:06,2006-08-15,1.379e+06,"MacArthur grant,simplicity,industrial design,a...","In terms of invention, I'd like to tell you th...",2006,8


In [104]:
Data2.tail()

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
2381,2797.0,Justin Davidson,Why glass towers are bad for city life -- and ...,http://www.ted.com/talks/view/id/2797,There's a creepy transformation taking over ou...,http://www.ted.com/talks/view/id/2797/transcri...,3.0,2017.0,TEDNYC,0:12:39,2017-06-06,519418.0,"architecture,beauty,cities,community,journalis...",Imagine that when you walked in here this even...,2017,6
2382,2798.0,Mehdi Ordikhani-Seyedlar,What happens in your brain when you pay attent...,http://www.ted.com/talks/view/id/2798,Attention isn't just about what we focus on i...,http://www.ted.com/talks/view/id/2798/transcri...,4.0,2017.0,TED2017,0:06:32,2017-06-08,422132.0,"AI,algorithm,brain,cognitive science,machine l...",Paying close attention to something: Not that ...,2017,6
2383,2799.0,Tim Ferriss,Why you should define your fears instead of yo...,http://www.ted.com/talks/view/id/2799,"The hard choices what we most fear doing, ask...",http://www.ted.com/talks/view/id/2799/transcri...,4.0,2017.0,TED2017,0:13:21,2017-06-12,582001.0,"choice,fear,goal-setting,humanity,life,persona...","So, this happy pic of me was taken in 1999. I ...",2017,6
2384,2801.0,Anne Lamott,12 truths I learned from life and writing,http://www.ted.com/talks/view/id/2801,"A few days before she turned 61, writer Anne L...",http://www.ted.com/talks/view/id/2801/transcri...,4.0,2017.0,TED2017,0:15:55,2017-06-09,651425.0,"Christianity,God,aging,art,beauty,birds,books,...",My seven-year-old grandson sleeps just down th...,2017,6
2385,2804.0,Richard Browning,How I built a jet suit,http://www.ted.com/talks/view/id/2804,We've all dreamed of flying but for Richard B...,http://www.ted.com/talks/view/id/2804/transcri...,4.0,2017.0,TED2017,0:07:08,2017-06-13,323733.0,"adventure,collaboration,demo,design,engineerin...","Michael Browning: engineer, innovator inventor...",2017,6


In [105]:
Data2.shape

(2386, 16)

### Check Speaker Details

In [106]:
Data2[Data2.speaker == "Amy Cuddy"]

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
1260,1569.0,Amy Cuddy,Your body language shapes who you are,http://www.ted.com/talks/view/id/1569,"Body language affects how others see us, but i...",http://www.ted.com/talks/view/id/1569/transcri...,6.0,2012.0,TEDGlobal 2012,0:21:02,2012-10-01,4.151e+07,"TED Brain Trust,success,body language,psycholo...",So I want to start by offering you a free no-t...,2012,10


### Check Speech Details with Index No.

In [107]:
Data2.iloc[1260,:]

id                                                               1.57e+03
speaker                                                         Amy Cuddy
headline                            Your body language shapes who you are
URL                                 http://www.ted.com/talks/view/id/1569
description             Body language affects how others see us, but i...
transcript_URL          http://www.ted.com/talks/view/id/1569/transcri...
month_filmed                                                            6
year_filmed                                                      2.01e+03
event                                                      TEDGlobal 2012
duration                                                          0:21:02
date_published                                        2012-10-01 00:00:00
views_as_of_06162017                                             4.15e+07
tags                    TED Brain Trust,success,body language,psycholo...
transcript              So I want to s

In [108]:
Data2.iloc[1260:1261,:]   #Above in DF format

,id,speaker,headline,URL,description,transcript_URL,month_filmed,year_filmed,event,duration,date_published,views_as_of_06162017,tags,transcript,Pub_Year,Pub_Month
1260,1569.0,Amy Cuddy,Your body language shapes who you are,http://www.ted.com/talks/view/id/1569,"Body language affects how others see us, but i...",http://www.ted.com/talks/view/id/1569/transcri...,6.0,2012.0,TEDGlobal 2012,0:21:02,2012-10-01,4.151e+07,"TED Brain Trust,success,body language,psycholo...",So I want to start by offering you a free no-t...,2012,10


### Check Transcript with Index No.

In [109]:
Transcripts = Data2['transcript']

In [110]:
Transcripts[1260]

'So I want to start by offering you a free no-tech life hack, and all it requires of you is this: that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you\'re doing with your body. So how many of you are sort of making yourselves smaller? Maybe you\'re hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out. I see you. So I want you to pay attention to what you\'re doing right now. We\'re going to come back to that in a few minutes, and I\'m hoping that if you learn to tweak this a little bit, it could significantly change the way your life unfolds. So, we\'re really fascinated with body language, and we\'re particularly interested in other people\'s body language. You know, we\'re interested in, like, you know an awkward interaction, or a smile, or a contemptuous glance, or maybe a very awkward wink, or maybe even something lik